In [1]:
import boto3
import requests
from bs4 import BeautifulSoup
import datetime
import numpy as np
import pandas as pd

In [2]:
results = requests.get("https://listings.loopghana.com/pageNumber_1")

In [3]:
results.headers

{'Server': 'nginx/1.16.1', 'Date': 'Tue, 26 Nov 2019 21:30:09 GMT', 'Content-Type': 'text/html;charset=UTF-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Content-Encoding': 'gzip'}

In [4]:
soup = BeautifulSoup(results.content, 'lxml')

In [5]:
def c(x):
    #If element is empty dont call the .text argument
    if x:
        return x.text
    else:
        return ''
    

In [7]:
soup.find_all('ul', {'class': 'listings-list'})

[<ul class="listings-list">
 </ul>, <ul class="listings-list">
 <li>
 <a data-id="c1o5" href="https://listings.loopghana.com/listing/c1o5">
 <img src="https://djwv1vs796km.cloudfront.net/thumbnails/media/3/61/916361_1572878958361.jpeg"/>
 <span class="name">  Cantonment </span>
 <div class="remove" title="Remove from Search Results"></div>
 <div class="favorite" title="Favorite this listing"></div>
 <span class="price">
 																								
 												
 						$2,500
 						
 												
 											</span>
 <span class="bedrooms">10 Bed</span> <span class="bathrooms">10 Bath</span>
 <span class="size">
 																					4,046
 														m<sup>2</sup>
 </span>
 <span class="agt">
 													Adsan Consult
 											</span>
 </a>
 </li>
 <li>
 <a data-id="c1d8" href="https://listings.loopghana.com/listing/c1d8">
 <img src="https://djwv1vs796km.cloudfront.net/thumbnails/media/3/14/916314_1572863418113.jpeg"/>
 <span class="name">  Cantonment </span>
 <div class="remove

In [35]:
clean = soup.find('span', {'class': 'price'}).text.replace('\t', '').replace('\r', '').replace('\n', '')

In [67]:
import re
curr = re.sub(r'(\d)+', r' \1', clean).split(' ')[0]

In [68]:
curr

'$'

In [ ]:
soup.find('span', {'class': 'bathrooms'}).text.split(" ")[0] ##This is for bathrooms
soup.find('span', {'class': 'name'}) ##For description

In [ ]:
def scrape_a_page(page):
    properties = []

    # property is a python reserved word so we'll use prop
    for prop in page.find_all("div", class_="mqs-prop-dt-wrapper"):
        name = prop.h2.a.text
#         location = name.split(' at ')[1].strip().split(' in ')[0]
        location = prop.find('h2').text.replace('@', ' at ').split(' at ')[1].strip().split(' in ')[0]
        rate = prop.find_all('p')[0].text
        
        currency, price, period = extract_rate(rate)

        # description = prop.find_all('p')[1].text
        # beds = prop.find('li', {'class': 'bed'}).text
        # showers = prop.find('li', {'class': 'shower'}).text
        # garages = prop.find('li', {'class': 'garages'}).text
        
        # This looks cryptic, but it's a way to handle pages that 
        # have missing info.

        description = c(prop.find_all('p')[1])
        beds = c(prop.find('li', {'class': 'bed'}))
        showers = c(prop.find('li', {'class': 'shower'}))
        garages = c(prop.find('li', {'class': 'garage'}))
        area = c(prop.find('li', {'class': 'area'})).replace(' m2', '').replace(',', '')
        url = prop.find('a').attrs['href']
        
        x = url.replace('/', '').split('-')
        apartment_type = x[0]
        rent_type = x[2]
        
        
        properties.append(
            {
                'property': name,
                'beds': beds,
                'showers': showers,
                'garages': garages,
                'area': area,
                'description': description,
                'price': price,
                'currency': currency,
                'rent_period': period,
                'url': url,
                'address': location,
                'apartment_type': apartment_type,
                'rent_type': rent_type,
                'time_posted': None
            }
        )

    return properties